# Multi-session analysis
This notebook demonstrate how to combine different xarray dataset together and perform group-level analysis

In [ ]:
%reload_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pylab as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import numpy as np
from glob import glob
import re
from datetime import datetime
from trialexp.process.folder_org.utils import build_session_info, load_and_concat_dataset, filter_sessions

## Build dataframe containing the metadata of all sessions

`build_session_info` will extract information from the by_sessions folder and build a dataframe for further filtering


In [ ]:
#specify the location of the by_sessions folder here
root_path = '/home/MRC.OX.AC.UK/ndcn1330/ettin/Julien/Data/head-fixed/by_sessions'

df_session_info = build_session_info(root_path)
df_session_info

## Loading data

`filter_sessions` provide a set of functions for filtering recording sessions. For details on how to use it, please consult its documentation

In [ ]:
#select the last 5 sessions from animal kms058
df_sel = filter_sessions(df_session_info, 'kms058', session_no=5, session_method='tail', task_name='reaching_go_spout_bar_nov22')
df_sel

`load_and_conat_dataset` is a helper functions that combines multiple sessions of dataset together

In [ ]:
paths = df_sel.path
ds_combined = load_and_concat_dataset(paths)

In [ ]:
ds_combined

## Plot figures

In [ ]:
df2plot = ds_combined[['hold_for_water_analog_1_df_over_f', 'success']].to_dataframe().reset_index()
sns.lineplot(x='event_time',hue='success',
            y='hold_for_water_analog_1_df_over_f', n_boot=100, data=df2plot)

In [ ]:
df2plot = ds_combined[['first_bar_off_analog_1_df_over_f', 'success']].to_dataframe().reset_index()
sns.lineplot(x='event_time',hue='success',
            y='first_bar_off_analog_1_df_over_f',n_boot=100,  data=df2plot)

In [ ]:
df2plot = ds_combined[['first_spout_analog_1_df_over_f', 'success']].to_dataframe().reset_index()
sns.lineplot(x='event_time',hue='success',
            y='first_spout_analog_1_df_over_f',n_boot=100,  data=df2plot)

### ACh

The following provides another examples on ACh data

In [ ]:
df_sel = filter_sessions(df_session_info, 'RE602', session_no=3, session_method='tail', task_name='reaching_go_spout_bar_nov22')
df_sel


In [ ]:
paths = df_sel.path
ds_ach_combined = load_and_concat_dataset(paths)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(4*3,3))

df2plot = ds_ach_combined[['hold_for_water_analog_1_df_over_f', 'success']].to_dataframe().reset_index()
sns.lineplot(x='event_time',hue='success',
            y='hold_for_water_analog_1_df_over_f', data=df2plot, ax=ax[0])

df2plot = ds_ach_combined[['first_bar_off_analog_1_df_over_f', 'success']].to_dataframe().reset_index()
sns.lineplot(x='event_time',hue='success',
            y='first_bar_off_analog_1_df_over_f', data=df2plot, ax=ax[1])

df2plot = ds_ach_combined[['first_spout_analog_1_df_over_f', 'success']].to_dataframe().reset_index()
sns.lineplot(x='event_time',hue='success',
            y='first_spout_analog_1_df_over_f', data=df2plot, ax=ax[2])

fig.tight_layout()